# Library

In [5]:
import pandas as pd
import numpy as np
from pycaret.classification import *

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import os

# Config

In [2]:
DATA_DIR = '../data/'
RAW_DIR = DATA_DIR + '01_raw/'

DATASET_DIR = 'titanic/'

INPUT_DIR = RAW_DIR + DATASET_DIR

In [6]:
OUTPUT_DIR = DATA_DIR + '05_model_input/' + DATASET_DIR
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [7]:
ID_COL = 'PassengerId'
Y_COL = 'Survived'

# Load data

In [8]:
# train
train_fname = INPUT_DIR + 'train.csv'
base_train_df = pd.read_csv(train_fname)
base_train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
# test
test_fname = INPUT_DIR + 'test.csv'
base_test_df = pd.read_csv(test_fname)
base_test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# Preprocessing

## Define method

In [24]:
def len_str(x):
    return len(str(x))

def left_str(x, num_chars):
    calib_num_chars = num_chars - 1
    return str(x)[:calib_num_chars]

## Minimum processing

In [25]:
train_df = base_train_df.copy().assign(flg='train')
test_df = base_test_df.copy().assign(flg='test')
test_df[Y_COL] = None

prepro_df = pd.concat([train_df, test_df])
prepro_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,flg
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,train
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,train
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,train
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,train
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,train


## Name

In [26]:
# 欠損値補間
prepro_df['fix_Name'] = prepro_df['Name'].fillna('Null')

# 文字数
prepro_df['len_Name'] = prepro_df['fix_Name'].map(len_str)

## Ticket

In [27]:
# 欠損値補間
prepro_df['fix_Ticket'] = prepro_df['Ticket'].fillna('-1')

# 文字数
prepro_df['len_Ticket'] = prepro_df['fix_Ticket'].map(len_str)

# 最初の2文字
prepro_df['first_chars_Ticket'] = prepro_df['fix_Ticket'].map(lambda x: left_str(x, 2))

## Cabin

In [28]:
# 欠損値補間
prepro_df['fix_Cabin'] = prepro_df['Cabin'].fillna('-1')

# 文字数
prepro_df['len_Cabin'] = prepro_df['fix_Cabin'].map(len_str)

# 最初の1文字
prepro_df['first_chars_Cabin'] = prepro_df['fix_Cabin'].map(lambda x: left_str(x, 1))

# Output

In [29]:
prepro_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,flg,fix_Name,len_Name,fix_Ticket,len_Ticket,first_chars_Ticket,fix_Cabin,len_Cabin,first_chars_Cabin
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,train,"Braund, Mr. Owen Harris",23,A/5 21171,9,A,-1,2,
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,train,"Cumings, Mrs. John Bradley (Florence Briggs Th...",51,PC 17599,8,P,C85,3,
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,train,"Heikkinen, Miss. Laina",22,STON/O2. 3101282,16,S,-1,2,
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,train,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",44,113803,6,1,C123,4,
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,train,"Allen, Mr. William Henry",24,373450,6,3,-1,2,


In [30]:
prepro_df.to_pickle(OUTPUT_DIR + 'train_test.pkl')